# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2443185004906,
 'week52high': 164.64,
 'week52low': 116.3,
 'week52highSplitAdjustOnly': 162.38,
 'week52lowSplitAdjustOnly': 117.9,
 'week52change': 0.25051920953426,
 'sharesOutstanding': 16660489560,
 'float': 0,
 'avg10Volume': 63844530,
 'avg30Volume': 71780346,
 'day200MovingAvg': 147.37,
 'day50MovingAvg': 148.27,
 'employees': 148211,
 'ttmEPS': 11.42,
 'ttmDividendRate': 0.8874165607198211,
 'dividendYield': 0.005942171577748288,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-22',
 'nextEarningsDate': '2022-01-20',
 'peRatio': 13.182204642729406,
 'beta': 1.390258483901489,
 'maxChangePercent': 58.28696881598067,
 'year5ChangePercent': 4.9181942316470595,
 'year2ChangePercent': 1.3224799446188875,
 'year1ChangePercent': 0.28652959966845537,
 'ytdChangePercent': 0.1236003156311743,
 'month6ChangePercent': 0.17456620367977854,
 'month3ChangePercent': 0.0180027562449588,
 'month1ChangePercent': 0.03706673897746394,
 'day30ChangePer

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.28652959966845537

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])



my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
#final_dataframe
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #from printing we see a lot of nested info, we grab price and stats endpoint
    #print(data['AAPL']['price']) <-- looking in here, we see where to get the exact values
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series( #this is to match up with the table columns
        [
            symbol,
            data[symbol]['price'], 
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
            
        ],
        index = my_columns),
            ignore_index = True
        )

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,161.260,0.454539,N/A
1,AAL,21.640,0.751416,N/A
2,AAP,245.050,0.534008,N/A
3,AAPL,152.547,0.287109,N/A
4,ABBV,116.950,0.241204,N/A
...,...,...,...,...
500,YUM,125.540,0.249734,N/A
501,ZBH,137.240,-0.104505,N/A
502,ZBRA,592.790,0.716184,N/A
503,ZION,66.980,0.721021,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True) #inplace replaces the table
final_dataframe = final_dataframe[:50] #i just want the top 50
final_dataframe.reset_index(inplace = True) #reset the index
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.68,2.753534,N/A
1,275,LB,83.72,2.366834,N/A
2,317,MRO,17.11,2.348798,N/A
3,175,FANG,113.49,2.311311,N/A
4,195,FTNT,340.79,2.052091,N/A
5,299,MCHP,84.96,1.729968,N/A
6,357,OXY,32.15,1.61647,N/A
7,111,COTY,11.07,1.473075,N/A
8,174,F,20.36,1.350423,N/A
9,345,NVDA,312.77,1.314241,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('enter portfolio size')
    try:
        float(portfolio_size)
    except ValueError:
        portfolio_size = input('enter a number')

portfolio_input()
print(portfolio_size)

enter portfolio size10000000
10000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
#print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe
#we've chosen the top 50 high movers and spent the money evenly between them
#no matter how much many stocks that means

/home/warrenball/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.68,2.753534,4578
1,275,LB,83.72,2.366834,2388
2,317,MRO,17.11,2.348798,11689
3,175,FANG,113.49,2.311311,1762
4,195,FTNT,340.79,2.052091,586
5,299,MCHP,84.96,1.729968,2354
6,357,OXY,32.15,1.61647,6220
7,111,COTY,11.07,1.473075,18066
8,174,F,20.36,1.350423,9823
9,345,NVDA,312.77,1.314241,639


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

#hqm_dataframe
#same logic as before, using batch calls for 
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series(
        [
            symbol, #'Ticker'
            data[symbol]['price'], #'Price'
            'N/A',
            data[symbol]['stats']['year1ChangePercent'],
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],
        index = hqm_columns),
        ignore_index = True
        )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.230,N/A,0.466617,N/A,0.216738,N/A,0.018955,N/A,0.045757,N/A,N/A
1,AAL,21.830,N/A,0.744128,N/A,-0.020406,N/A,0.031232,N/A,0.075114,N/A,N/A
2,AAP,247.730,N/A,0.544772,N/A,0.139259,N/A,0.134326,N/A,0.097978,N/A,N/A
3,AAPL,151.421,N/A,0.29641,N/A,0.174641,N/A,0.017464,N/A,0.037869,N/A,N/A
4,ABBV,117.360,N/A,0.248196,N/A,0.029278,N/A,0.029356,N/A,0.065626,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,124.730,N/A,0.248337,N/A,0.041245,N/A,-0.051611,N/A,0.020397,N/A,N/A
501,ZBH,136.670,N/A,-0.105032,N/A,-0.225379,N/A,-0.080764,N/A,-0.089984,N/A,N/A
502,ZBRA,605.740,N/A,0.72055,N/A,0.209526,N/A,0.026103,N/A,0.16999,N/A,N/A
503,ZION,67.140,N/A,0.730453,N/A,0.126219,N/A,0.200227,N/A,0.032461,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'    
]
#we're saving these values from the main list to help

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.fillna(0,inplace=True)# need this if batch call gives any n/a
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        #score = percentileofscore, put in column and value
        hqm_dataframe.loc[row, percentile_col] = score( hqm_dataframe[change_col] , hqm_dataframe.loc[row, change_col] )/100
        #divide by 100 to make it a percentage
        #print(type(hqm_dataframe[f'{time_period} Price Return']))
        #print((hqm_dataframe.loc[row, f'{time_period} Price Return']))
        #BE Careful!! in four of these calls, there are nontypes rather than floats, the print statements above reveal it
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.230,N/A,0.466617,0.700990,0.216738,0.853465,0.018955,0.437624,0.045757,0.49901,N/A
1,AAL,21.830,N/A,0.744128,0.883168,-0.020406,0.308911,0.031232,0.493069,0.075114,0.689109,N/A
2,AAP,247.730,N/A,0.544772,0.772277,0.139259,0.708911,0.134326,0.839604,0.097978,0.764356,N/A
3,AAPL,151.421,N/A,0.296410,0.465347,0.174641,0.790099,0.017464,0.429703,0.037869,0.455446,N/A
4,ABBV,117.360,N/A,0.248196,0.425743,0.029278,0.467327,0.029356,0.477228,0.065626,0.629703,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,124.730,N/A,0.248337,0.427723,0.041245,0.495050,-0.051611,0.166337,0.020397,0.358416,N/A
501,ZBH,136.670,N/A,-0.105032,0.039604,-0.225379,0.031683,-0.080764,0.095050,-0.089984,0.027723,N/A
502,ZBRA,605.740,N/A,0.720550,0.871287,0.209526,0.839604,0.026103,0.469307,0.169990,0.948515,N/A
503,ZION,67.140,N/A,0.730453,0.879208,0.126219,0.689109,0.200227,0.936634,0.032461,0.419802,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

#now we grab the means of all the return percentiles

for row in hqm_dataframe.index:
    momentum_percentiles = [] #save all percentile scores in this list and grab the mean of all of it
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.230,N/A,0.466617,0.700990,0.216738,0.853465,0.018955,0.437624,0.045757,0.49901,0.622772
1,AAL,21.830,N/A,0.744128,0.883168,-0.020406,0.308911,0.031232,0.493069,0.075114,0.689109,0.593564
2,AAP,247.730,N/A,0.544772,0.772277,0.139259,0.708911,0.134326,0.839604,0.097978,0.764356,0.771287
3,AAPL,151.421,N/A,0.296410,0.465347,0.174641,0.790099,0.017464,0.429703,0.037869,0.455446,0.535149
4,ABBV,117.360,N/A,0.248196,0.425743,0.029278,0.467327,0.029356,0.477228,0.065626,0.629703,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,124.730,N/A,0.248337,0.427723,0.041245,0.495050,-0.051611,0.166337,0.020397,0.358416,0.361881
501,ZBH,136.670,N/A,-0.105032,0.039604,-0.225379,0.031683,-0.080764,0.095050,-0.089984,0.027723,0.048515
502,ZBRA,605.740,N/A,0.720550,0.871287,0.209526,0.839604,0.026103,0.469307,0.169990,0.948515,0.782178
503,ZION,67.140,N/A,0.730453,0.879208,0.126219,0.689109,0.200227,0.936634,0.032461,0.419802,0.731188


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]# We now sort based on how high the HQM score is
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,314.060,N/A,1.329721,0.982178,1.081147,0.998020,0.494028,0.994059,0.423618,1.0,0.993564
1,F,19.560,N/A,1.333687,0.984158,0.662221,0.986139,0.418427,0.992079,0.285649,0.992079,0.988614
2,ANET,537.930,N/A,1.059009,0.954455,0.621964,0.980198,0.416073,0.990099,0.418340,0.99802,0.980693
3,AMD,152.730,N/A,0.830374,0.920792,0.846525,0.996040,0.319879,0.974257,0.345431,0.99604,0.971782
4,SPG,167.180,N/A,1.211626,0.970297,0.327763,0.938614,0.258624,0.964356,0.250603,0.982178,0.963861
5,DXCM,654.570,N/A,0.851273,0.932673,0.846360,0.994059,0.245915,0.958416,0.179546,0.954455,0.959901
6,MCHP,86.650,N/A,1.755893,0.990099,1.319336,1.000000,1.211002,1.000000,0.112890,0.825743,0.95396
7,LYV,119.200,N/A,0.799401,0.906931,0.390331,0.962376,0.388607,0.984158,0.177608,0.952475,0.951485
8,KLAC,413.340,N/A,0.838872,0.924752,0.371398,0.956436,0.187473,0.928713,0.244828,0.980198,0.947525
9,XLNX,213.830,N/A,0.644490,0.831683,0.670907,0.988119,0.358882,0.978218,0.271203,0.988119,0.946535


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

enter portfolio size10000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
#position_size

for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price']) 

hqm_dataframe

/home/warrenball/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,314.060,636,1.329721,0.982178,1.081147,0.998020,0.494028,0.994059,0.423618,1.0,0.993564
1,F,19.560,10224,1.333687,0.984158,0.662221,0.986139,0.418427,0.992079,0.285649,0.992079,0.988614
2,ANET,537.930,371,1.059009,0.954455,0.621964,0.980198,0.416073,0.990099,0.418340,0.99802,0.980693
3,AMD,152.730,1309,0.830374,0.920792,0.846525,0.996040,0.319879,0.974257,0.345431,0.99604,0.971782
4,SPG,167.180,1196,1.211626,0.970297,0.327763,0.938614,0.258624,0.964356,0.250603,0.982178,0.963861
5,DXCM,654.570,305,0.851273,0.932673,0.846360,0.994059,0.245915,0.958416,0.179546,0.954455,0.959901
6,MCHP,86.650,2308,1.755893,0.990099,1.319336,1.000000,1.211002,1.000000,0.112890,0.825743,0.95396
7,LYV,119.200,1677,0.799401,0.906931,0.390331,0.962376,0.388607,0.984158,0.177608,0.952475,0.951485
8,KLAC,413.340,483,0.838872,0.924752,0.371398,0.956436,0.187473,0.928713,0.244828,0.980198,0.947525
9,XLNX,213.830,935,0.644490,0.831683,0.670907,0.988119,0.358882,0.978218,0.271203,0.988119,0.946535


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column( f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0] , column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save() #oddly enough, it won't update in your system, you must download the excel file